# Voltage clamp and currents

In [ ]:
!pip install neuron

     |████████████████████████████████| 12.6MB 226kB/s 


In [ ]:
from neuron import h
from neuron.units import mV, ms, µm
h.load_file("stdrun.hoc")

1.0

In [ ]:
import plotly
import plotly.graph_objects as go

In [ ]:
soma = h.Section(name="soma")
soma.L = soma.diam = 10 * µm

In [ ]:
h.hh.insert(soma)

# Voltage clamp at 0 mV

In [ ]:
vclamp = h.SEClamp(soma(0.5))
vclamp.amp1 = -65 * mV
vclamp.dur1 = 5 * ms
vclamp.dur2 = 20 * ms
vclamp.amp2 = 20 * mV
vclamp.amp3 = -65 * mV
vclamp.dur3 = 10000 * ms

In [ ]:
t = h.Vector().record(h._ref_t)
ina = h.Vector().record(soma(0.5)._ref_ina)
ik = h.Vector().record(soma(0.5)._ref_ik)

In [ ]:
h.finitialize(-65 * mV)
h.continuerun(30 * ms)

0.0

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=ina, name="ina"))
fig.add_trace(go.Scatter(x=t, y=ik, name="ik"))


# Many voltage clamp experiments

In [ ]:
k_fig = go.Figure()
na_fig = go.Figure()

for v in [-80, -70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40]:
  vclamp.amp2 = v
  h.finitialize(-65 * mV)
  h.continuerun(30 * ms)
  k_fig.add_trace(go.Scatter(x=t, y=ik, name=f"v={v}", line={"width": 4}))
  na_fig.add_trace(go.Scatter(x=t, y=ina, name=f"v={v}", line={"width": 4}))

k_fig.update_layout(dict(
    xaxis_title="t (ms)",
    yaxis_title="ik (mA/cm**2)",
    title="Voltage clamp experiments: Potassium"
))

na_fig.update_layout(dict(
    xaxis_title="t (ms)",
    yaxis_title="ina (mA/cm**2)",
    title="Voltage clamp experiments: Sodium"
))


k_fig.show()
na_fig.show()

# Let's look at potassium concentration

Here the voltage clamp is at `v=40 mV` because that's where it ended above, but the result would be identical for all choices.

Why? Hodgkin and Huxley's model didn't include dynamic changes to ion concentrations (they assumed homeostasis), although if we assume no homeostatic mechanisms, then the change in concentration follows from Avogadro.

In [ ]:
ki = h.Vector().record(soma(0.5)._ref_ki)
ko = h.Vector().record(soma(0.5)._ref_ko)
h.finitialize(-65 * mV)
h.continuerun(30 * ms)

0.0

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=ki, name="ki"))
fig.add_trace(go.Scatter(x=t, y=ko, name="ko"))

# Allowing dynamic potassium concentrations

Chemical dynamics are enabled using the rxd module. Here we define the cytosol (by default it covers the entire cross-section but in reality it would be some portion) and register the potassium ion in there, and now when we rerun, we see changes to the concentrations. (NB: this overrides the assumption of homeostasis for Potassium, and any such mechanisms must now be explicitly modeled.)

We could similarly look at extracellular concentrations in 1 (Frankenhauser-Hodgkin space) or 3 dimensions.

In [ ]:
from neuron import rxd

cyt = rxd.Region(h.allsec(), name="cyt", nrn_region="i")
k = rxd.Species(cyt, name="k", charge=1)

In [ ]:
h.finitialize(-65 * mV)
h.continuerun(30 * ms)

0.0

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=ki, name="ki"))
fig.update_layout(dict(
    xaxis_title="t (ms)",
    yaxis_title="ki (mM)",
    title="Intracellular potassium concentration over time"
))
fig.show()